<a href="https://colab.research.google.com/github/Coduck-repo/fun-fun-stock/blob/master/NaverStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import urllib.parse
from urllib.request import urlopen
from bs4 import BeautifulSoup

# 종목코드 가져오기
stock_code = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

# 보합 https://finance.naver.com/sise/sise_steady.nhn?sosok=1
# 상승 https://finance.naver.com/sise/sise_rise.nhn?sosok=1
kospi_url = "https://finance.naver.com/sise/sise_steady.nhn?sosok=0"
kosdaq_url = "https://finance.naver.com/sise/sise_steady.nhn?sosok=1"


def GetSteadyStock(type):
    if (type == 'kospi'):
        url = kospi_url
    else:
        url = kosdaq_url

    html = urlopen(url)
    content = html.read().decode('euc-kr', 'replace').encode('utf-8', 'replace')
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find("table", class_="type_2")
    sise = []
    for tr in table.find_all("tr")[1:]:
        tds = tr.find_all("td")
        stock = []
        for td in tds[1:10]:
            value = td.get_text().strip().replace(',', '')
            try:
                number = int(value)  # 수치 변환
                stock.append(number)
            except ValueError:  # 문자열
                stock.append(value)
        if len(stock):
            sise.append(tuple(stock))

    df = pd.DataFrame(sise, columns=['종목명', '현재가', '전일비', '등락률', '거래량', '매수호가', '매도호가', '매수총잔량', '매도총잔량'])
    steady_stock = pd.merge(df, stock_code.rename(columns= {'회사명':'종목명'})[['종목명','종목코드']])
    return steady_stock

kospi_df = GetSteadyStock('kospi')
kosdaq_df = GetSteadyStock('kosdaq')

# 정렬 (거래량)
#df1 = df1.sort_values(by='거래량', ascending=False)

# 종목토론실 게시물 컨텐츠
# 페이지 크롤링 필요함
def StockBoardContent(code):
    url = "https://finance.naver.com/item/board.nhn?code=%06d" % code

    board = []
    html = urlopen(url)
    content = html.read().decode('euc-kr', 'replace').encode('utf-8', 'replace')
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find("table", class_="type2")
    for tr in table.find_all("tr")[1:]:
        item = []
        tds = tr.find_all("td")
        for td in tds:
            value = td.get_text().strip().replace('\t','').replace('\t','').replace('\n','')
            item.append(value)
        
        if len(item) != 1:
          link = urllib.parse.parse_qs(tds[1].find('a')['href'])
          board.append(tuple(link['nid']) + tuple(item))

    df = pd.DataFrame(board, columns=['nid', '날짜','제목','투자의견','글쓴이','조회','공감','비공감'])
    return df

# 전 종목 게시판 크롤링
#for index, row in kospi_df.iterrows():
#    df = StockBoardCount(row['종목코드'])

In [0]:
df = StockBoardContent(990)

In [56]:
df.sort_values(by='날짜', ascending=False)

,nid,날짜,제목,투자의견,글쓴이,조회,공감,비공감
0,114890783,2019.12.17 19:51,주가,의견없음,ccw1****,12,0,0
8,114890679,2019.12.17 19:48,잘먹고 갑니다.,의견없음,hong****,3,0,0
2,114890197,2019.12.17 19:33,인텔이 하바나랩스 인수했네 ...,의견없음,suk7****,215,3,1
1,114890221,2019.12.17 19:33,뭐지???[3],의견없음,nnur****,230,6,1
3,114889447,2019.12.17 19:10,주식[1],의견없음,dnsk****,376,7,3
5,114889014,2019.12.17 18:58,잡상인임~클릭금지,의견없음,kgbs****,200,1,0
4,114888886,2019.12.17 18:54,수익 내신 분들 축하드립니다,의견없음,hand****,459,7,2
6,114888667,2019.12.17 18:45,이런주식에 기사 하나안쓰는 기관투자자들 ...,의견없음,a880****,528,7,1
7,114888284,2019.12.17 18:33,잘먹고 갑니다.,의견없음,ctac****,492,5,1
18,114888259,2019.12.17 18:32,낼은 상[1],의견없음,9999****,140,1,0
